<b> Dataquest </b> - Data Scientist / Working with Data Sources / SQL and Databases: Intermediate / Guided Project: Preparing data for SQLite 
- Author: Florian Kaiser
- Date: 01/08/17
- Goal: Prepare "academy_awards.csv" data set to read into SQLite

## 1. Import dataset

In [124]:
import pandas as pd
awards = pd.read_csv("academy_awards.csv", encoding='ISO-8859-1')
awards.head(2)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Filtering and cleaning up dataset

In [125]:
# Clean up year column by by selecting just the first 4 digits in each value in the column
awards["Year"] = awards.Year.str[0:4].astype('int64')

In [126]:
# Create new data frame by selecting only rows where Year is larger than 2000
later_than_2000 = awards[awards.Year > 2000]

In [127]:
# Create list of targeted award categories
award_categories = ["Actor -- Leading Role","Actor -- Supporting Role","Actress -- Leading Role","Actress -- Supporting Role"]
for a in award_categories:
    print (a)

Actor -- Leading Role
Actor -- Supporting Role
Actress -- Leading Role
Actress -- Supporting Role


In [128]:
# Create df nominations by filtering for list of target award_categories
nominations = later_than_2000[later_than_2000.Category.isin(award_categories)]
nominations.Category.value_counts()

Actor -- Supporting Role      50
Actor -- Leading Role         50
Actress -- Leading Role       50
Actress -- Supporting Role    50
Name: Category, dtype: int64

In [129]:
# Convert "Won?" column to numeric values by mappin No/Yes to 0/1
replace_dict = { "NO": 0, "YES": 1 }
nominations["Won"] = nominations["Won?"].map(replacements).copy()

/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [130]:
# Drop extraneous columns and assign to final_nominations df
drop_columns = ["Won?","Unnamed: 5","Unnamed: 6","Unnamed: 7","Unnamed: 8","Unnamed: 9","Unnamed: 10"]
final_nominations = nominations.drop(drop_columns, axis=1, inplace=False)

In [131]:
# Extract movie title as well as character and assign it to two seperate columns 
final_nominations["Additional_info_new"] = final_nominations["Additional Info"].str.rstrip("'}").str.split(" {'")
final_nominations["Movie"]= final_nominations.Additional_info_new.str[0]
final_nominations["Character"]= final_nominations.Additional_info_new.str[1]

In [132]:
# Drop extraneous columns to create final version of final_nominations
final_nominations.drop(["Additional Info","Additional_info_new"], axis=1, inplace=True)

In [133]:
final_nominations.head(2)

,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn


## 3. Exporting to SQLite

In [134]:
# Donnect to database file nominations.db 
import sqlite3
conn = sqlite3.connect("nominations.db")

In [135]:
# Export final_nominations to nominations.db
final_nominations.to_sql("nominations", conn, index=False)

ValueError: Table 'nominations' already exists.

## 4. Verify database functionality

In [ ]:
# Explore database by printing db-schema
query1 = """
pragma table_info(nominations)
;
"""
print(conn.execute(query1).fetchall())

In [ ]:
# Print first 10 rows of nominations table
query2 = """
SELECT *
FROM nominations
LIMIT 10
;
"""
print(conn.execute(query1).fetchall())

In [ ]:
# Close database connection
conn.close()